In [52]:
fname = './data/train.csv'

In [53]:
import csv

In [151]:
import json
json_fname = './data/letter-recognition.json'


In [152]:
with open(json_fname, 'r') as file:
    data = json.load(file)

In [163]:
keys = list(map(int, data.keys()))

In [173]:
validation = {k: data[k] for k in list(map(str, sorted(keys)[-2000:]))}

In [177]:
train = {k: data[k] for k in map(str, range(1, len(keys)-2000))}

In [180]:
with open('./data/train.json', 'w') as f:
    json.dump(train, f)
    
with open('./data/validation.json', 'w') as f:
    json.dump(validation, f)

In [243]:
fname = './data/validation.json'

with open(fname, 'r') as f:
    data = json.load(f)

n_examples = len(data.keys())
labels = np.ndarray((n_examples,))

label_to_int = dict()
label_counter = 0
for i, v in enumerate(data.values()):
    this_label = [x for x in v.keys()][0]
    if this_label not in label_to_int.keys():
        label_to_int[this_label] = label_counter
        label_counter += 1
    labels[i] = label_to_int[this_label]
    
    if i == 0:
        feat_order = list(v[this_label].keys())
        feats = np.ndarray((n_examples, len(feat_order)))
    
    for j, o in enumerate(feat_order):
        feats[i,j] = v[this_label][o]
        
    

In [284]:
feats.shape

(2000, 16)

## Import data
### To numpy features and labels

In [51]:
import numpy as np

In [62]:
feats = np.loadtxt(fname, skiprows=1, usecols=[l for l in range(1, 17)], delimiter=',')

In [64]:
labels = np.loadtxt(fname, skiprows=1, usecols=0, delimiter=',')

ValueError: could not convert string to float: b'T'

## Multinomial Naive Bayes

In [182]:
n_classes = 26
n_feats = 16

In [229]:
from collections import defaultdict
collector = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))

feats_list = []
labels_list = []
with open(fname, newline='') as csvfile:
    reader = csv.reader(csvfile)
    data = list(reader)

num_examples = len(data)
header = data[0]
feats = np.array([row[1:] for row in data[1:]], dtype=int)
label_chars = [row[0] for row in data[1:]]
char2int = dict(zip(sorted(set(label_chars)), range(len(set(label_chars)))))
labels = np.array([char2int[c] for c in label_chars])


In [232]:
num_classes = len(set(label_chars))
num_feats = feats.shape[1]
print(num_examples, num_classes, num_feats)

14990 26 16


In [240]:
feat_range = range(15)
for i in range(num_classes):
    this_class = feats[np.where(labels==i)]
    num_in_class = this_class.shape[0]
    for j in feat_range:
        print((np.sum(tmp==j, axis=0) / num_in_class)

[ 0.00165017  0.03960396  0.          0.01485149  0.00660066  0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.        ]
[ 0.06105611  0.04290429  0.02310231  0.05610561  0.05940594  0.          0.
  0.          0.          0.          0.          0.          0.0709571   0.
  0.          0.        ]
[ 0.12706271  0.03135314  0.06105611  0.06930693  0.09240924  0.          0.
  0.04785479  0.00330033  0.          0.          0.          0.4950495   0.
  0.0049505   0.        ]
[ 0.20957096  0.0660066   0.10891089  0.07425743  0.15676568  0.          0.
  0.15841584  0.02640264  0.          0.03630363  0.          0.11551155
  0.00165017  0.00990099  0.00165017]
[ 0.24257426  0.04785479  0.18976898  0.15181518  0.1980198   0.          0.
  0.1369637   0.10231023  0.          0.08415842  0.          0.04620462
  0.0049505   0.06105611  0.0049505 ]
[ 0.14356436  0.07590759  0.20792079  0.10891089  0.13036304  0.
  0.02475248  0.217

In [105]:
#     for l, row in enumerate(reader):
#         if l == 0:
#             header = row
#         else:
#             # array approach (for Gaussian)
#             labels_list.append(row[0])
#             feats_list.append(row[1:])
            
#             # count dictionary approach (for multinomial)
#             y = row[0]
#             for c, el in enumerate(row[1:]):
#                 collector[y][c][int(el)] += 1

In [120]:
# Compute P(X1 = x1 | Y = y)
# For inference, compute P(Y = y | X1 = x1, X2 = x2, ...) as 
# argmax{ P(X1=x1|Y=y)*P(X2=x2|Y=y)*...}


def prob(collector, feat_idx, feat_val, y):
    
    # smoothing params
    n_feats = 16
    alpha = 1
    z = sum(collector[y][feat_idx].values()) + alpha*n_feats
    return (alpha+collector[y][feat_idx][feat_val]) / z
    

In [146]:
# Training error
n_examples = len(feats_list)
n_classes = 26
all_probs = np.ndarray((n_examples, n_classes))
for row in feats_list:
    for j, y in enumerate(sorted(collector.keys())):
        p = 1
        for i, x in enumerate(row):
            p = p * prob(collector, i, int(x), y)
        all_probs[j, i]



In [139]:
labels

array([19,  3, 13, ..., 17, 19, 16])

In [147]:
predict = np.argmax(all_probs,axis=1)

In [148]:
sum(labels==predict)/len(labels)

0.75201814664086997

In [112]:
feat_val_keys = set()
for y in collector.keys():
    for i in collector[y].keys():
        feat_val_keys.update(collector[y][i].keys())

In [113]:
feat_val_keys

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15}

In [124]:
collector.keys()

dict_keys(['D', 'M', 'S', 'T', 'B', 'X', 'W', 'V', 'A', 'Z', 'O', 'J', 'E', 'G', 'C', 'K', 'H', 'Y', 'R', 'U', 'I', 'N', 'F', 'L', 'Q', 'P'])

In [122]:
for y in collector.keys():
    for feat_idx in range(16):
        for feat_val in range(15):
            p = prob(collector, feat_idx, feat_val, y)
            if p == 0:
                print(y, feat_idx, feat_val, p)

## Gaussian Naive Bayes

In [75]:
feats = np.array(feats_list, dtype=int)

In [84]:
keys = set(labels_list)
label_map = dict(zip(sorted(keys), range(len(keys))))
labels = np.array([label_map[label] for label in labels_list])

In [85]:
print(feats.shape, labels.shape)

(14989, 16) (14989,)


In [205]:
import math
def gaussprob(means, stds, feat_vals, y):
    m = means[y]
    s = stds[y]
    p = np.exp(-0.5*np.power((feat_vals - m)/s, 2)) * (1/(s*math.sqrt(2*math.pi)))
    return np.sum(np.log(p))

In [200]:
feat_vals = feats[0]
m = means[0]
s = stds[0]

In [198]:
np.power((feat_vals - m)/s, 2)

array([  7.96451454e-01,   8.86417740e-02,   1.94649676e+00,
         9.67322175e-03,   1.26223266e+00,   1.82448122e-01,
         2.59697441e+01,   4.40887800e+00,   1.03003819e+01,
         1.08960333e+00,   1.75234944e+01,   1.71201607e-01,
         4.34496820e+00,   1.97409858e+00,   3.10979907e+00,
         1.28210527e-01])

In [211]:
predict = np.ndarray(n_examples)
for i in range(n_examples):
    predict[i] = np.argmax([gaussprob(means, stds, feats[i], c) for c in range(n_classes)])

In [213]:
sum(labels==predict)/len(labels)

0.6530789245446661

In [188]:
eps = 1e-6
means = np.zeros([n_classes, n_feats])
stds = np.zeros([n_classes, n_feats]) + eps

for v in label_map.values():
    v_feats = feats[np.where(labels == v)]
    print(v_feats.shape)
    means[v] = (np.mean(v_feats, axis=0))
    stds[v] = (np.std(v_feats, axis=0))


(603, 16)
(607, 16)
(553, 16)
(589, 16)
(572, 16)
(580, 16)
(576, 16)
(573, 16)
(606, 16)
(548, 16)
(567, 16)
(556, 16)
(571, 16)
(575, 16)
(553, 16)
(542, 16)
(547, 16)
(623, 16)
(580, 16)
(597, 16)
(564, 16)
(584, 16)
(582, 16)
(594, 16)
(560, 16)
(587, 16)
